<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/SQL_Spark_with_OLD_version_JoseDianes_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>
<hr>

![alt text](http://1.bp.blogspot.com/-nqAGzznZQNo/UwS8rxjfXeI/AAAAAAAABTA/nunmRLowpps/s1600/PraxisLogo.gif)<br>
[Data Science Program](http://praxis.ac.in/Programs/business-analytics/)

<hr>

[Prithwis Mukerjee](http://www.yantrajaal.com)<br>

# Jose A Dianes Tutorial


---

https://www.codementor.io/jadianes/spark-python-rdd-basics-du107x2ra <br>
https://www.codementor.io/jadianes/spark-python-data-aggregations-du107on3m <br>
https://github.com/jadianes/spark-py-notebooks -- notebooks <br>
https://github.com/prithwis/spark-py-notebooks -- fork of above 

# Install Spark

In [1]:
!apt-get update 
!apt-get install openjdk-8-jdk-headless 

!wget  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz   # Using Older Version 
#!wget -q https://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz  # Current version does not work

#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#


!tar xf spark-3.0.3-bin-hadoop3.2.tgz
#!tar xf spark-3.2.1-bin-hadoop3.2.tgz



#!pip3 install -qq pyspark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 k

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [3]:
!pip install -q findspark
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Load Data

In [5]:
#import urllib.request
#urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz','kdd10.gz')
#urllib.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kdd10b.gz")
!wget 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'

--2022-05-25 07:18:22--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘kddcup.data_10_percent.gz’

kddcup.data_10_perc 100%[===================>]   2.04M  9.13MB/s    in 0.2s    

2022-05-25 07:18:22 (9.13 MB/s) - ‘kddcup.data_10_percent.gz’ saved [2144903/2144903]



In [6]:
!rm kdd10.gz
!mv kddcup.data_10_percent.gz kdd10.gz

rm: cannot remove 'kdd10.gz': No such file or directory


In [ ]:
data_file = "./kdd10.gz"
raw_data = sc.textFile(data_file)

print ("Train data size is ", raw_data.count())

Train data size is  494021


In [ ]:
#import urllib.request
#urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz','corrected.gz')
!wget 'http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz'

--2022-05-20 06:16:10--  http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409035 (1.3M) [application/x-gzip]
Saving to: ‘corrected.gz’

corrected.gz        100%[===================>]   1.34M  3.79MB/s    in 0.4s    

2022-05-20 06:16:10 (3.79 MB/s) - ‘corrected.gz’ saved [1409035/1409035]



In [ ]:
test_data_file = "./corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print ("Test data size is ", test_raw_data.count())

Test data size is  311029


# Part 10 SQL Dataframes <br>
https://github.com/prithwis/spark-py-notebooks/blob/master/nb10-sql-dataframes/nb10-sql-dataframes.ipynb

In [7]:
data_file = "./kdd10.gz"
raw_data = sc.textFile(data_file)

print ("Train data size is ", raw_data.count())

Train data size is  494021


In [8]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)

In [9]:
raw_data.collect()[1]

'0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.'

In [ ]:
#SQLContext is deprecated
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

In [10]:
from pyspark.sql import Row

# -------------------------------------------------------
# Using the full data set causes the SparkSQL to hang up
# Have not been able to find a solution to this problem
# -------------------------------------------------------

# However, if we use an OLDER version of Spark, the error goes away


csv_data = raw_data.map(lambda l: l.split(","))            # using full data will cause problem with spark 3.2.1 will work with 3.0.3
#csv_data = raw_data_sample.map(lambda l: l.split(","))      # using 10% sample data
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [16]:
csv_data.take(1)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

In [15]:
row_data.take(2)

[Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=181, dst_bytes=5450),
 Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=239, dst_bytes=486)]

In [12]:
#interactions_df = sqlContext.createDataFrame(row_data) -- deprecated
interactions_df = spark.createDataFrame(row_data)
#interactions_df.registerTempTable("interactions") -- deprecated
interactions_df.createOrReplaceTempView("interactions")

In [13]:
#tcp_interactions = sqlContext.sql("""
tcp_interactions = spark.sql("""
    SELECT distinct(protocol_type) FROM interactions 
""")
tcp_interactions.show()

+-------------+
|protocol_type|
+-------------+
|          tcp|
|          udp|
|         icmp|
+-------------+



In [17]:
#tcp_interactions = sqlContext.sql("""
tcp_interactions = spark.sql("""
    SELECT  count(*) FROM interactions 
""")
tcp_interactions.show()

+--------+
|count(1)|
+--------+
|  494021|
+--------+



In [18]:
#tcp_interactions = sqlContext.sql("""
tcp_interactions = spark.sql("""
    SELECT duration, protocol_type, dst_bytes FROM interactions WHERE protocol_type = 'udp' 
""")
tcp_interactions.show()

+--------+-------------+---------+
|duration|protocol_type|dst_bytes|
+--------+-------------+---------+
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       8|          udp|       32|
|       7|          udp|       36|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|        0|
|       0|          udp|       32|
|       4|          udp|       41|
|       0|          udp|       75|
|       0|          udp|       48|
|       0|          udp|       48|
|       0|          udp|       48|
+--------+-------------+---------+
only showing top 20 rows



In [19]:
#cp_interactions = sqlContext.sql("""
tcp_interactions = spark.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [20]:
# Output duration together with dst_bytes
tcp_interactions_out = tcp_interactions.rdd.map(lambda p: ("Duration = ", p.duration,"DST Bytes = ", p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print (ti_out)

('Duration = ', 5057, 'DST Bytes = ', 0)
('Duration = ', 5059, 'DST Bytes = ', 0)
('Duration = ', 5051, 'DST Bytes = ', 0)
('Duration = ', 5056, 'DST Bytes = ', 0)
('Duration = ', 5051, 'DST Bytes = ', 0)
('Duration = ', 5039, 'DST Bytes = ', 0)
('Duration = ', 5062, 'DST Bytes = ', 0)
('Duration = ', 5041, 'DST Bytes = ', 0)
('Duration = ', 5056, 'DST Bytes = ', 0)
('Duration = ', 5064, 'DST Bytes = ', 0)
('Duration = ', 5043, 'DST Bytes = ', 0)
('Duration = ', 5061, 'DST Bytes = ', 0)
('Duration = ', 5049, 'DST Bytes = ', 0)
('Duration = ', 5061, 'DST Bytes = ', 0)
('Duration = ', 5048, 'DST Bytes = ', 0)
('Duration = ', 5047, 'DST Bytes = ', 0)
('Duration = ', 5044, 'DST Bytes = ', 0)
('Duration = ', 5063, 'DST Bytes = ', 0)
('Duration = ', 5068, 'DST Bytes = ', 0)
('Duration = ', 5062, 'DST Bytes = ', 0)
('Duration = ', 5046, 'DST Bytes = ', 0)
('Duration = ', 5052, 'DST Bytes = ', 0)
('Duration = ', 5044, 'DST Bytes = ', 0)
('Duration = ', 5054, 'DST Bytes = ', 0)
('Duration = ', 

In [21]:
interactions_df.printSchema()

root
 |-- duration: long (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)



In [22]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()
tt = time() - t0

print ("Query performed in ",round(tt,3)," seconds")

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Query performed in  7.365  seconds


In [23]:
t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").filter(interactions_df.duration>1000).filter(interactions_df.dst_bytes==0).groupBy("protocol_type").count().show()
tt = time() - t0

print ("Query performed in ",round(tt,3)," seconds")

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Query performed in  7.113  seconds


In [24]:
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"
    
row_labeled_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    label=get_label_type(p[41])
    )
)
#interactions_labeled_df = sqlContext.createDataFrame(row_labeled_data)
interactions_labeled_df = spark.createDataFrame(row_labeled_data)

In [25]:
t0 = time()
interactions_labeled_df.select("label").groupBy("label").count().show()
tt = time() - t0

print ("Query performed in ",round(tt,3)," seconds")

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Query performed in  6.944  seconds


In [26]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type").groupBy("label", "protocol_type").count().show()
tt = time() - t0

print ("Query performed in ",round(tt,3)," seconds")

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Query performed in  6.628  seconds


In [27]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type", "dst_bytes").groupBy("label", "protocol_type", interactions_labeled_df.dst_bytes==0).count().show()
tt = time() - t0

print ("Query performed in ",round(tt,3)," seconds")

+------+-------------+---------------+------+
| label|protocol_type|(dst_bytes = 0)| count|
+------+-------------+---------------+------+
|normal|          udp|          false| 15583|
|attack|          udp|          false|    11|
|attack|          tcp|           true|110583|
|normal|          tcp|          false| 67500|
|attack|         icmp|           true|282314|
|attack|          tcp|          false|  2669|
|normal|          tcp|           true|  9313|
|normal|          udp|           true|  3594|
|normal|         icmp|           true|  1288|
|attack|          udp|           true|  1166|
+------+-------------+---------------+------+

Query performed in  6.855  seconds


In [28]:
#interactions_df.registerTempTable("interactions") -- deprecated
interactions_labeled_df.createOrReplaceTempView("interactions_l")

In [29]:
#tcp_interactions = sqlContext.sql("""
tcp_interactions2 = spark.sql("""
    SELECT * FROM interactions_l limit 20
""")
tcp_interactions2.show()

+--------+-------------+-------+----+---------+---------+------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes| label|
+--------+-------------+-------+----+---------+---------+------+
|       0|          tcp|   http|  SF|      181|     5450|normal|
|       0|          tcp|   http|  SF|      239|      486|normal|
|       0|          tcp|   http|  SF|      235|     1337|normal|
|       0|          tcp|   http|  SF|      219|     1337|normal|
|       0|          tcp|   http|  SF|      217|     2032|normal|
|       0|          tcp|   http|  SF|      217|     2032|normal|
|       0|          tcp|   http|  SF|      212|     1940|normal|
|       0|          tcp|   http|  SF|      159|     4087|normal|
|       0|          tcp|   http|  SF|      210|      151|normal|
|       0|          tcp|   http|  SF|      212|      786|normal|
|       0|          tcp|   http|  SF|      210|      624|normal|
|       0|          tcp|   http|  SF|      177|     1985|normal|
|       0|          tcp| 

In [38]:
#tcp_interactions = sqlContext.sql("""
tcp_interactions2 = spark.sql("""
    SELECT  label, protocol_type,  dst_bytes==0 check0, count(*) count FROM interactions_l group by label, protocol_type, check0
""")
tcp_interactions2.show()

+------+-------------+------+------+
| label|protocol_type|check0| count|
+------+-------------+------+------+
|normal|          udp| false| 15583|
|attack|          udp| false|    11|
|attack|          tcp|  true|110583|
|normal|          tcp| false| 67500|
|attack|         icmp|  true|282314|
|attack|          tcp| false|  2669|
|normal|          tcp|  true|  9313|
|normal|          udp|  true|  3594|
|normal|         icmp|  true|  1288|
|attack|          udp|  true|  1166|
+------+-------------+------+------+



#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)